## Reflection Architecture

In [13]:
from dotenv import load_dotenv
import os

load_dotenv()
hf_api_token = os.getenv("HF_API_TOKEN")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [14]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a senior AI researcher tasked with writing excellent prompts to complete tasks."
            " Generate the best prompt possible to instruct a language model to complete the task specified by the user."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatOpenAI(
        api_key=openai_api_key,
        temperature=0.5,
        model='gpt-4o'
    )
generate = prompt | llm

In [16]:
prompt = ""
request = HumanMessage(
    content="Write a function to calculate the factorial of a number."
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    prompt += chunk.content

Certainly! To create a function that calculates the factorial of a number, the prompt should be clear, concise, and include all necessary details. Here’s an ideal prompt:

---

**Prompt:**

Write a Python function named `factorial` that takes a single integer parameter `n` and returns the factorial of `n`. The factorial of a non-negative integer `n` is the product of all positive integers less than or equal to `n`. For example, the factorial of 5 (denoted as 5!) is 5 * 4 * 3 * 2 * 1 = 120. Ensure that the function handles edge cases such as the factorial of 0, which is defined to be 1. Include error handling to manage negative inputs by returning `None` or raising an appropriate exception.

**Example Usage:**

```python
print(factorial(5))  # Output: 120
print(factorial(0))  # Output: 1
print(factorial(-3)) # Output: None or raise ValueError
```

---

This prompt provides clear instructions and examples to guide the language model in generating the desired function.

### Reflection

In [17]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert prompt engineer evaluating a prompt. Generate critique and recommendations for the user's prompt."
            " Provide detailed recommendations, including requests for intructions, context, examples, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [19]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=prompt)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Your initial prompt is clear but lacks some details that can help ensure the function is implemented correctly and robustly. Here are some specific recommendations to improve it:

1. **Function Name and Parameters:**
   - Clearly specify the function name and its parameters.
   
2. **Definition and Examples:**
   - Provide a clear definition of what a factorial is.
   - Include examples of expected input and output.

3. **Edge Cases:**
   - Mention how to handle edge cases, such as the factorial of 0.
   - Specify how to handle negative inputs (e.g., return `None` or raise an exception).

4. **Programming Language:**
   - Specify the programming language (if not obvious).

5. **Error Handling:**
   - Explicitly state how to handle invalid inputs.

Here’s an improved version of the prompt:

---

**Prompt:**

Write a Python function named `factorial` that calculates the factorial of a given non-negative integer `n`. The function should:

1. Take a single integer parameter `n`.
2. Return 

In [20]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=prompt), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

Thank you for your detailed feedback! Based on your recommendations, here’s the revised prompt incorporating all the specified improvements:

---

**Prompt:**

Write a Python function named `factorial` that calculates the factorial of a given non-negative integer `n`. The function should:

1. Take a single integer parameter `n`.
2. Return the factorial of `n`, which is the product of all positive integers less than or equal to `n`.
3. Return `1` if `n` is `0`, as the factorial of `0` is defined to be `1`.
4. Handle negative inputs by either returning `None` or raising a `ValueError` with an appropriate error message.

**Definition:**
The factorial of a non-negative integer `n` (denoted as `n!`) is the product of all positive integers less than or equal to `n`. For example:
- `5! = 5 * 4 * 3 * 2 * 1 = 120`
- `3! = 3 * 2 * 1 = 6`
- `0! = 1`

**Example Usage:**

```python
def factorial(n):
    if not isinstance(n, int):
        raise TypeError("Input must be an integer.")
    if n < 0:
  

### Graph

In [21]:
from typing import List, Sequence

from langgraph.graph import END, MessageGraph


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")


def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [26]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Generate a prompt to write a function to calculate the factorial of a number."
        )
    ],
):
    print(event)
    print("---")

{'generate': AIMessage(content='Sure! Here is a prompt to instruct a language model to write a function to calculate the factorial of a number:\n\n---\n\n**Task: Write a Function to Calculate the Factorial of a Number**\n\n**Instructions:**\n\n1. **Function Definition**: Define a function named `calculate_factorial` that takes a single argument, `n`, which is a non-negative integer.\n2. **Factorial Concept**: The factorial of a non-negative integer `n` is the product of all positive integers less than or equal to `n`. It is denoted as `n!`.\n   - For example: \n     - `0! = 1`\n     - `1! = 1`\n     - `5! = 5 * 4 * 3 * 2 * 1 = 120`\n3. **Edge Case**: Ensure that the function correctly handles the edge case where `n` is 0. By definition, `0!` is `1`.\n4. **Implementation**: Use a loop or recursion to calculate the factorial.\n5. **Return Value**: The function should return the factorial of the given number.\n\n**Example Usage:**\n```python\nprint(calculate_factorial(5))  # Output: 120\n